In [8]:
import numpy as np

from two_step_classification import *
print_version()

python:      3.10.11
sklearn:     1.2.2
tensorflow:  2.12.0
keras:       2.12.0
numpy:       1.23.5
pandas:      1.5.3


In [9]:
# load data
x_train: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_train_df.pkl")
x_test: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_test_df.pkl")
y_train: pd.Series = pd.read_pickle("models/kdd99_features/y_train_df.pkl")
y_test: pd.Series = pd.read_pickle("models/kdd99_features/y_test_df.pkl")
x_train_ae_48: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_train+ae_48_df&activation=relu&epochs=5&batch_size=32.pkl")
x_test_ae_48: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_test+ae_48_df&activation=relu&epochs=5&batch_size=32.pkl")

In [10]:
lr_binary_dir = 'models/logistic_regression_binary/'
lr_anomaly_dir = 'models/logistic_regression_anomaly/'
lgb_binary_dir = 'models/lightgbm/'


In [11]:
#
with open("models/logistic_regression_binary/kdd99+ae_48&penalty=l1&solver=liblinear&C=0.1.pkl", 'rb') as fp:
    model_1st: LogisticRegression = pickle.load(fp)
with open("models/logistic_regression_anomaly/kdd99+ae_48&penalty=l1&solver=liblinear&C=0.1.pkl", 'rb') as fp:
    model_2nd: LogisticRegression = pickle.load(fp)
two_step_classification(x_test_ae_48, y_test, model_1st, model_2nd, verbose=2)


0    129201
2      1310
3       326
4         9
dtype: int64
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     32102
           1       1.00      1.00      1.00    130925

    accuracy                           1.00    163027
   macro avg       1.00      1.00      1.00    163027
weighted avg       1.00      1.00      1.00    163027

              precision    recall  f1-score   support

         dos       1.00      1.00      1.00    129181
      normal       0.99      1.00      1.00     32102
       probe       0.97      0.94      0.96      1355
         r2l       0.89      0.78      0.83       372
         u2r       0.78      0.41      0.54        17

    accuracy                           1.00    163027
   macro avg       0.93      0.83      0.86    163027
weighted avg       1.00      1.00      1.00    163027

              normal  anomaly
true_normal    31983      119
true_anomaly     198   130727
                dos  normal  prob

(              normal  anomaly
 true_normal    31983      119
 true_anomaly     198   130727,
                 dos  normal  probe  r2l  u2r
 true_dos     129137      42      0    2    0
 true_normal      53   31983     31   34    1
 true_probe       10      69   1276    0    0
 true_r2l          1      77      3  290    1
 true_u2r          0      10      0    0    7)

In [12]:
model_1st: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_binary_tuned_booster.model")
model_2nd: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_tuned_booster.model")
two_step_classification(x_test, y_test, model_1st, model_2nd, verbose=2)

0    129186
2      1335
3       352
1        25
4        12
dtype: int64
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32102
           1       1.00      1.00      1.00    130925

    accuracy                           1.00    163027
   macro avg       1.00      1.00      1.00    163027
weighted avg       1.00      1.00      1.00    163027

              precision    recall  f1-score   support

         dos       1.00      1.00      1.00    129181
      normal       1.00      1.00      1.00     32102
       probe       1.00      0.98      0.99      1355
         r2l       0.99      0.94      0.96       372
         u2r       0.83      0.59      0.69        17

    accuracy                           1.00    163027
   macro avg       0.96      0.90      0.93    163027
weighted avg       1.00      1.00      1.00    163027

              normal  anomaly
true_normal    32089       13
true_anomaly      28   130897
                dos  

(              normal  anomaly
 true_normal    32089       13
 true_anomaly      28   130897,
                 dos  normal  probe  r2l  u2r
 true_dos     129175       6      0    0    0
 true_normal       4   32093      1    3    1
 true_probe        6      15   1334    0    0
 true_r2l          1      21      0  349    1
 true_u2r          0       7      0    0   10)

In [14]:
pd.Series(np.argmax(model_2nd.predict(x_train), axis=1)).value_counts()

0    262277
1     65176
2      2752
3       754
4        35
dtype: int64

In [16]:
model_1st: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb+ae_48_binary_tuned_booster.model")
model_2nd: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb+ae_48_anomaly_tuned_booster.model")

In [17]:
two_step_classification(x_test_ae_48, y_test, model_1st, model_2nd, verbose=2)

0    93692
2    36834
3      357
4       33
dtype: int64
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32102
           1       1.00      1.00      1.00    130925

    accuracy                           1.00    163027
   macro avg       1.00      1.00      1.00    163027
weighted avg       1.00      1.00      1.00    163027

              precision    recall  f1-score   support

         dos       1.00      0.73      0.84    129181
      normal       1.00      1.00      1.00     32102
       probe       0.04      0.99      0.07      1355
         r2l       0.99      0.95      0.97       372
         u2r       0.36      0.71      0.48        17

    accuracy                           0.78    163027
   macro avg       0.68      0.87      0.67    163027
weighted avg       0.99      0.78      0.87    163027

              normal  anomaly
true_normal    32088       14
true_anomaly      23   130902
               dos  normal  probe  r2

(              normal  anomaly
 true_normal    32088       14
 true_anomaly      23   130902,
                dos  normal  probe  r2l  u2r
 true_dos     93690       5  35481    2    3
 true_normal      2   32088      9    1    2
 true_probe       0       8   1342    2    3
 true_r2l         0       5      2  352   13
 true_u2r         0       5      0    0   12)